In [36]:
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer

ds = load_dataset("rajpurkar/squad")

model = AutoModelForCausalLM.from_pretrained("../distillLLAMA2")
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")

In [37]:
import pandas as pd

train_dataset=ds["train"].shuffle(seed=42).select(range(1000))

In [83]:
i=6

In [84]:
train_dataset[i]

{'id': '57324e38e17f3d1400422823',
 'title': 'Dwight_D._Eisenhower',
 'context': 'Following the German unconditional surrender, Eisenhower was appointed Military Governor of the U.S. Occupation Zone, based at the IG Farben Building in Frankfurt am Main. He had no responsibility for the other three zones, controlled by Britain, France and the Soviet Union, except for the city of Berlin, which was managed by the Four-Power Authorities through the Allied Kommandatura as the governing body. Upon discovery of the Nazi concentration camps, he ordered camera crews to document evidence of the atrocities in them for use in the Nuremberg Trials. He reclassified German prisoners of war (POWs) in U.S. custody as Disarmed Enemy Forces (DEFs), who were no longer subject to the Geneva Convention. Eisenhower followed the orders laid down by the Joint Chiefs of Staff (JCS) in directive JCS 1067, but softened them by bringing in 400,000 tons of food for civilians and allowing more fraternization. In res

In [118]:
tokenized = tokenizer(train_dataset[i]['context']+train_dataset[i]['question'] + train_dataset[i]['answers']['text'][0])
cqlen = len(tokenizer(train_dataset[i]['context']+train_dataset[i]['question'])['input_ids'])
alen = tokenizer(train_dataset[i]['answers']['text'][0])

In [121]:
cqlen

253

In [123]:
alen

{'input_ids': [128000, 86426, 24245, 315, 279, 549, 815, 13, 99452, 22967], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [122]:
len(tokenized['input_ids'])

262

In [126]:
tokenized['input_ids'][261]

22967

In [124]:
tokenized

{'input_ids': [128000, 28055, 279, 6063, 84663, 32859, 11, 89181, 574, 21489, 28058, 24245, 315, 279, 549, 815, 13, 99452, 22967, 11, 3196, 520, 279, 47280, 13759, 8123, 17283, 304, 56284, 1097, 4802, 13, 1283, 1047, 912, 12014, 369, 279, 1023, 2380, 26467, 11, 14400, 555, 13527, 11, 9822, 323, 279, 19953, 9323, 11, 3734, 369, 279, 3363, 315, 20437, 11, 902, 574, 9152, 555, 279, 13625, 9483, 1223, 68415, 1555, 279, 61855, 27790, 1969, 25282, 439, 279, 10217, 2547, 13, 30538, 18841, 315, 279, 32527, 20545, 29135, 11, 568, 11713, 6382, 43160, 311, 2246, 6029, 315, 279, 72336, 304, 1124, 369, 1005, 304, 279, 452, 554, 76, 7881, 70544, 13, 1283, 312, 65974, 6063, 28240, 315, 4208, 320, 2089, 75833, 8, 304, 549, 815, 13, 25502, 439, 4185, 31372, 31851, 30500, 320, 13963, 82, 705, 889, 1051, 912, 5129, 3917, 311, 279, 45345, 26958, 13, 89181, 8272, 279, 10373, 17551, 1523, 555, 279, 30833, 45844, 315, 17381, 320, 41, 6546, 8, 304, 34959, 622, 6546, 220, 7461, 22, 11, 719, 90158, 1124, 555, 1

In [117]:
tokenizer.decode(30)

'?'

In [134]:
labels=[-100]*(cqlen-1)
for i in range(len(tokenized['input_ids'])-cqlen):
    labels.append(tokenized['input_ids'][cqlen+i])
labels.append(-100)

In [137]:
tokenized['labels']=labels

In [138]:
tokenized

{'input_ids': [128000, 28055, 279, 6063, 84663, 32859, 11, 89181, 574, 21489, 28058, 24245, 315, 279, 549, 815, 13, 99452, 22967, 11, 3196, 520, 279, 47280, 13759, 8123, 17283, 304, 56284, 1097, 4802, 13, 1283, 1047, 912, 12014, 369, 279, 1023, 2380, 26467, 11, 14400, 555, 13527, 11, 9822, 323, 279, 19953, 9323, 11, 3734, 369, 279, 3363, 315, 20437, 11, 902, 574, 9152, 555, 279, 13625, 9483, 1223, 68415, 1555, 279, 61855, 27790, 1969, 25282, 439, 279, 10217, 2547, 13, 30538, 18841, 315, 279, 32527, 20545, 29135, 11, 568, 11713, 6382, 43160, 311, 2246, 6029, 315, 279, 72336, 304, 1124, 369, 1005, 304, 279, 452, 554, 76, 7881, 70544, 13, 1283, 312, 65974, 6063, 28240, 315, 4208, 320, 2089, 75833, 8, 304, 549, 815, 13, 25502, 439, 4185, 31372, 31851, 30500, 320, 13963, 82, 705, 889, 1051, 912, 5129, 3917, 311, 279, 45345, 26958, 13, 89181, 8272, 279, 10373, 17551, 1523, 555, 279, 30833, 45844, 315, 17381, 320, 41, 6546, 8, 304, 34959, 622, 6546, 220, 7461, 22, 11, 719, 90158, 1124, 555, 1